In [26]:
! pip install -U spacy -q 

In [27]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.8.2                         
Location         c:\Users\kingu\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy
Platform         Windows-11-10.0.26100-SP0     
Python version   3.12.7                        
Pipelines        pt_core_news_lg (3.8.0)       



In [28]:
# !wget https://raw.githubusercontent.com/amrrs/custom-ner-with-spacy/main/pvr_training_data.json

In [29]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.load("IaAntigo") 
db = DocBin() # create a DocBin object

In [30]:
import json
with open('training_data.json', encoding='utf-8') as f:
    TRAIN_DATA = json.load(f)

In [31]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 500/500 [00:00<00:00, 1508.16it/s]


In [32]:
! python -m spacy init config config.cfg --lang pt --pipeline ner --optimize efficiency


✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.



In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

In [ ]:
nlp_ner = spacy.load("model-best") 

In [ ]:
doc = nlp_ner('''Meu tablet, que foi dado pelo Governo do Estado do Ceará, não desativa mais o modo de rotação de tela, já tentei de todos os jeitos. Queria saber um modo de resolver isso.''') # input sample text

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter